Total params: 4,491,895

Training Accuracy: 99.91%

Validation Accuracy: 94.64%

Test Accuracy: 95.31%

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import shutil
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
BATCH_SIZE = 128
# You may need to lower this depending upon your GPU

TARGET_SIZE = (224, 224)
# This model uses 224x224 images

In [3]:
# These were created as part of our data preparation notebook
train_storage_ds = "C:/colab/data/birds/cache/train"
valid_storage_ds = "C:/colab/data/birds/cache/valid"
test_storage_ds = "C:/colab/data/birds/cache/test"

In [4]:
# Generator
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    brightness_range=[0.75,1.15],
    rotation_range=30,
    channel_shift_range=10,
    width_shift_range=1.0,
    height_shift_range=1.0,
    shear_range=0.02,
    horizontal_flip=True,
    fill_mode='reflect',
)

# Many folks are against validation augmentation but I use it as the changes
# here are mostly minimal and gives more variation. The test generator is pure.
validation_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    brightness_range=[0.75,1.15],
    rotation_range=30,
    channel_shift_range=10,
    fill_mode='reflect',
    validation_split=0.2,
    horizontal_flip=True,
)


test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

In [5]:
train_images = train_generator.flow_from_directory(
    directory=train_storage_ds,
    interpolation='bilinear',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
)

val_images = validation_generator.flow_from_directory(
    directory=valid_storage_ds,
    interpolation='bilinear',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
)

test_images = test_generator.flow_from_directory(
    directory=test_storage_ds,
    interpolation='bilinear',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 54067 images belonging to 524 classes.
Found 13516 images belonging to 524 classes.
Found 16896 images belonging to 524 classes.


In [6]:
# Visualize the images
def visualize_generator(generator, samples = 5):
    batch_images, labels = next(generator)
    plt.figure(figsize=(10,5))
    for i in range(samples):
        plt.subplot(1, samples, i+1)
        plt.imshow(batch_images[i] / 255.0)
        plt.axis('off')
    plt.show()
    batch_images, labels = next(generator)
    plt.figure(figsize=(10,5))
    for i in range(samples):
        plt.subplot(1, samples, i+1)
        plt.imshow(batch_images[i+5] / 255.0)
        plt.axis('off')
    plt.show()
    batch_images, labels = next(generator)
    plt.figure(figsize=(10,5))
    for i in range(samples):
        plt.subplot(1, samples, i+1)
        plt.imshow(batch_images[i+10] / 255.0)
        plt.axis('off')
    plt.show()

print("Train images:")
visualize_generator(train_images)

print("Validation images:")
visualize_generator(val_images)

print("Test images:")
visualize_generator(test_images)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
# absolute-cosine regularization
# https://www.amazon.science/publications/quantization-aware-training-with-absolute-cosine-regularization-for-automatic-speech-recognition
class ACosR(tf.keras.regularizers.Regularizer):
    def __init__(self, weight=1.0):
        self.weight = weight

    def __call__(self, weights):
        # Calculate the absolute cosine similarity
        cos_similarity = tf.reduce_mean(tf.abs(tf.reduce_sum(weights * tf.roll(weights, shift=1, axis=-1), axis=-1)))

        # Apply the regularization term to the loss
        regularization_loss = self.weight * cos_similarity

        return regularization_loss

    def get_config(self):
        return {'weight': float(self.weight)}


In [8]:
# Load the pretrained model
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
pretrained_model.trainable = False
pretrained_model.layers[2]._name = 'normalization'

# Enable ACosR
for i in range(0, len(pretrained_model.layers)):
    layer = pretrained_model.layers[i]
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.DepthwiseConv2D):
        pretrained_model.layers[i].kernel_regularizer = ACosR(weight = 0.005)

x = pretrained_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(200, trainable = False, kernel_regularizer = ACosR(weight = 0.005), name="dense_1", activation='relu')(x)
x = Dense(256, trainable = False, kernel_regularizer = ACosR(weight = 0.005), name="dense_2", activation='relu')(x)
outputs = Dense(524, kernel_regularizer = ACosR(weight = 0.005), trainable = False, activation=tf.keras.activations.softmax, name="dense_output")(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)

model.load_weights('C:/colab/data/birds/saves/224x224_524_full_swish/saved_model')


model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [9]:
# All layers are locked but running an epoch for demonstration purposes
model.compile(
    optimizer=Adam(1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=1,
)

423/423 [==============================] - 396s 927ms/step - loss: 0.0084 - accuracy: 0.9991 - val_loss: 0.3281 - val_accuracy: 0.9464


In [10]:
# Now let's evaluate the test images against this checkpoint
results = model.evaluate(test_images, verbose=1)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

132/132 [==============================] - 26s 198ms/step - loss: 0.2906 - accuracy: 0.9531
    Test Loss: 0.29056
Test Accuracy: 95.31%


In [11]:
# Let's now run predictions against the test images and show the output
# for a random subset as a grid
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

# Map the label
labels = (test_images.class_indices)
labels = dict((v, k) for k, v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f'The first 5 predictions: {pred[:5]}')

# Display 25 random pictures from the dataset with their labels
random_index = np.random.randint(0, len(test_images.filepaths), 25)
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(25, 25), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_images.filepaths[random_index[i]]))
    true_label = labels[test_images.classes[random_index[i]]]
    predicted_label = pred[random_index[i]]
    if true_label == predicted_label:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {true_label}\nPredicted: {predicted_label}", color=color)

plt.tight_layout()
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [12]:
# Classification report
y_test = [labels[idx] for idx in test_images.classes]
print(classification_report(y_test, pred))

                               precision    recall  f1-score   support

              ABBOTTS BABBLER       0.77      0.91      0.83        33
                ABBOTTS BOOBY       0.81      0.94      0.87        36
   ABYSSINIAN GROUND HORNBILL       0.97      0.97      0.97        36
        AFRICAN CROWNED CRANE       1.00      1.00      1.00        27
       AFRICAN EMERALD CUCKOO       0.96      0.84      0.90        31
            AFRICAN FIREFINCH       1.00      0.93      0.96        27
       AFRICAN OYSTER CATCHER       0.97      1.00      0.98        31
        AFRICAN PIED HORNBILL       0.94      0.92      0.93        37
          AFRICAN PYGMY GOOSE       1.00      1.00      1.00        36
                    ALBATROSS       1.00      0.96      0.98        26
               ALBERTS TOWHEE       0.91      1.00      0.96        32
         ALEXANDRINE PARAKEET       0.97      0.97      0.97        33
                ALPINE CHOUGH       0.94      1.00      0.97        33
     

In [13]:
# https://keras.io/examples/vision/grad_cam/

def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        inputs = model.inputs, outputs = [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    # Save the superimposed image
    superimposed_img.save(cam_path)
    return cam_path

preprocess_input = tf.keras.applications.efficientnet.preprocess_input
decode_predictions = tf.keras.applications.efficientnet.decode_predictions

last_conv_layer_name = "top_conv"
img_size = (224, 224, 3)

# Remove last layer's softmax
model.layers[-1].activation = None

In [14]:
# Let's now run predictions against the test images and show the output
# for a random subset as a grid
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

# Map the label
labels = (test_images.class_indices)
labels = dict((v, k) for k, v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f'The first 5 predictions: {pred[:5]}')

# Display 25 random pictures from the dataset with their labels
random_index = np.random.randint(0, len(test_images.filepaths), 25)
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(25, 25), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_images.filepaths[random_index[i]]))
    true_label = labels[test_images.classes[random_index[i]]]
    predicted_label = pred[random_index[i]]
    if true_label == predicted_label:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {true_label}\nPredicted: {predicted_label}", color=color)

plt.tight_layout()
plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [16]:
# Display the part of the pictures used by the neural network to classify the pictures
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img_path = test_images.filepaths[random_index[i]]
    img_array = tf.keras.applications.efficientnet.preprocess_input(get_img_array(img_path, size=img_size))
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    cam_path = save_and_display_gradcam(img_path, heatmap)
    ax.imshow(plt.imread(cam_path))
    ax.set_title(f"True: {labels[test_images.classes[random_index[i]]]}\nPredicted: {pred[random_index[i]]}")
plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.